In [1]:
import os

In [2]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [ ]:
filename = "..\\GoogleAPIKey.txt"

In [4]:
os.environ['GOOGLE_API_KEY'] = get_file_contents(filename)

In [ ]:
filename_grokKey = "..\\GroqAPIKey.txt"

In [6]:
os.environ['GROQ_API_KEY'] = get_file_contents(filename_grokKey)

### StructuredOutputParser - Output as a specified Schema

In [7]:
from langchain import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_groq import ChatGroq

In [8]:
#gemini_llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro-001')
lamma_llm = ChatGroq(model="llama-3.1-8b-instant")

In [10]:
response_schemas = [
    ResponseSchema(name="word",description="word entered by user"),
    ResponseSchema(name="meaning",description="One line meaning of the word given by user"),
    ResponseSchema(name="example",description="An example of how that word can be used in a line"),
    ResponseSchema(name="Etymology",description="Origin or history of the word"),
]

In [11]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"word": string  // word entered by user
	"meaning": string  // One line meaning of the word given by user
	"example": string  // An example of how that word can be used in a line
	"Etymology": string  // Origin or history of the word
}
```


In [12]:
prompt = PromptTemplate(
    input_variables=["word"],
    partial_variables={"format_instructions":format_instructions},
    template = "Provide the meaning, an example of how the  word is used in a sentence and the etymology of this word: {word}. \n {format_instructions}"
)

In [13]:
chain = prompt | lamma_llm | output_parser
chain.invoke({"Onomatopoeia"})

{'word': 'Onomatopoeia',
 'meaning': "A word that phonetically imitates, resembles or suggests the sound that it describes, such as 'buzz', 'meow', or 'splash'.",
 'example': 'The onomatopoeic effect of the sizzling bacon filled the kitchen with a mouth-watering sound.',
 'Etymology': "From Greek 'onomatopoiia', derived from 'onoma' (name) and 'poieo' (to make). The term was first used in English in 1656 to describe words that imitate sounds."}